In [1]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

import operator
import random

import numpy

from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, 
    smin=None, smax=None, best=None)

def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
    part[:] = list(map(operator.add, part, part.speed))

toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)
toolbox.register("evaluate", benchmarks.h1)

def main():
    pop = toolbox.population(n=5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 1000
    best = None

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
    
    return pop, logbook, best

if __name__ == "__main__":
    main()


gen	evals	avg      	std      	min      	max     
0  	5    	0.0834126	0.0259627	0.0366029	0.107688
1  	5    	0.0601908	0.0278188	0.0220512	0.089771
2  	5    	0.0764938	0.0545746	0.0287412	0.182684
3  	5    	0.0847159	0.0174277	0.0605406	0.110816
4  	5    	0.113452 	0.0415406	0.0641464	0.170965
5  	5    	0.103376 	0.065962 	0.0282223	0.225361
6  	5    	0.0906695	0.0155446	0.070683 	0.110119
7  	5    	0.0951116	0.0670203	0.0174322	0.182969
8  	5    	0.106582 	0.0397605	0.0578867	0.152715
9  	5    	0.0885431	0.0328085	0.0582574	0.146353
10 	5    	0.101193 	0.0297787	0.0587459	0.13224 
11 	5    	0.120914 	0.0455892	0.0687196	0.191205
12 	5    	0.148627 	0.0495456	0.0785519	0.217629
13 	5    	0.103425 	0.0613217	0.017646 	0.197332
14 	5    	0.13329  	0.0570007	0.0393607	0.193705
15 	5    	0.106928 	0.0751022	0.032275 	0.198884
16 	5    	0.14328  	0.0548401	0.0684042	0.194986
17 	5    	0.177531 	0.0668933	0.0658922	0.23862 
18 	5    	0.145063 	0.0404091	0.0850103	0.194968
19 	5    	0.0966592	